# Projet M2 MIAS : Risque de réidentification (Février 2026)

**Auteur :** [Votre Nom]
**Superviseur :** Pr Emmanuel Chazard

## 1. Objectif du projet
Conformément au sujet, l'objectif est d'imaginer et tester des indicateurs quantitatifs du risque de réidentification (Inférence, Corrélation, Individualisation) sur des bases de données synthétiques[cite: 3, 30].

Nous analyserons l'évolution du risque en fonction :
1. Du **niveau de mélange** (shuffling) des données.
2. De la **quantité d'informations** disponibles (scénarios de connaissance croissante).
3. De la nature de la base (**Directe** vs **Échantillonnée**).

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from datetime import datetime, timedelta

# Configuration pour la reproductibilité
np.random.seed(2026)
random.seed(2026)
plt.style.use('seaborn-v0_8')

## 2. Simulation des Données (Connaissances Externes)

Comme les fichiers sources ne sont pas fournis physiquement, nous générons la base `connaissances_externes.txt` en respectant strictement la description des variables du PDF [cite: 6-18].

**Variables générées :**
* `id_sejour` : Identifiant (pour vérification uniquement)[cite: 9].
* `age`, `age5`, `age10` : Précisions croissantes[cite: 10].
* `sexe`[cite: 11].
* Dates (`ymd`, `ym`, `y`) pour entrée et sortie[cite: 13, 14].
* Variables médicales (Target) : `liste_diag`, `liste_acte` (variables barrées/inconnues à inférer)[cite: 8, 17, 18].

In [ ]:
def generate_dataset(n_rows=1000):
    data = {}
    # 1. Identifiant
    data['id_sejour'] = np.arange(1, n_rows + 1)
    
    # 2. Démographie avec précisions croissantes
    ages = np.random.randint(0, 100, n_rows)
    data['age'] = ages
    data['age5'] = (ages // 5) * 5  # Arrondi à 5 ans
    data['age10'] = (ages // 10) * 10 # Arrondi à 10 ans
    data['sexe'] = np.random.choice(['M', 'F'], n_rows)
    
    # 3. Dates avec précisions croissantes
    start_date = datetime(2020, 1, 1)
    dates_entree = [start_date + timedelta(days=np.random.randint(0, 365*3)) for _ in range(n_rows)]
    durees = np.random.randint(1, 20, n_rows)
    dates_sortie = [d + timedelta(days=int(dur)) for d, dur in zip(dates_entree, durees)]
    
    # Formatage Entrée
    data['entree_date_ymd'] = [d.strftime('%Y-%m-%d') for d in dates_entree]
    data['entree_date_ym'] = [d.strftime('%Y-%m') for d in dates_entree]
    data['entree_date_y'] = [d.strftime('%Y') for d in dates_entree]
    
    # Formatage Sortie
    data['sortie_date_ymd'] = [d.strftime('%Y-%m-%d') for d in dates_sortie]
    data['sortie_date_ym'] = [d.strftime('%Y-%m') for d in dates_sortie]
    data['sortie_date_y'] = [d.strftime('%Y') for d in dates_sortie]
    
    # 4. Modes et Spécialités
    data['entree_mode'] = np.random.choice(['Urgence', 'Prog', 'Transfert'], n_rows)
    data['sortie_mode'] = np.random.choice(['Domicile', 'Transfert', 'Deces'], n_rows)
    data['specialite'] = np.random.choice(['MCO', 'CHIR', 'PSY', 'OBST'], n_rows)
    data['chirurgie'] = np.random.choice([0, 1], n_rows)
    
    # 5. Pathologies (Variables sensibles à inférer)
    # Ces variables correspondent aux "variables barrées" du sujet (CIM10, CCAM, pathologies)
    # Pour simplifier l'inférence, on utilise une cible binaire 'diabete'
    data['diabete'] = np.random.choice([0, 1], n_rows, p=[0.9, 0.1])
    data['liste_diag'] = [f"CIM_{np.random.randint(100, 999)}" for _ in range(n_rows)]
    
    df = pd.DataFrame(data)
    
    # Mélange aléatoire de l'ordre des colonnes (sauf id_sejour pour lisibilité) [cite: 20]
    cols = list(df.columns)
    cols.remove('id_sejour')
    np.random.shuffle(cols)
    return df[['id_sejour'] + cols]

df_reference = generate_dataset(2000)
print("Aperçu de connaissances_externes.txt :")
display(df_reference.head(3))

## 3. Génération des fichiers "out_direct" et "out_sample"

Nous implémentons ici la logique de perturbation décrite dans le sujet :
1.  **out_direct_XX** : Même population, mais XX% des cellules de chaque colonne sont mélangées.
2.  **out_sample_XX** : Tirage avec remise (bootstrap), puis mélange de XX% des cellules.

*Note : L'identifiant `id_sejour` est conservé pour vérifier le succès de la réidentification, mais ne sera pas utilisé comme clé de jointure par l'attaquant[cite: 28].*

In [ ]:
def apply_shuffle(df_input, percentage):
    """Mélange XX% des valeurs pour chaque colonne indépendamment."""
    df = df_input.copy()
    n_rows = len(df)
    n_shuffle = int(n_rows * (percentage / 100))
    
    if n_shuffle == 0:
        return df
        
    for col in df.columns:
        if col == 'id_sejour': continue  # On ne mélange pas l'ID pour pouvoir vérifier le résultat
        
        # Sélection des indices à mélanger
        indices_to_shuffle = np.random.choice(df.index, n_shuffle, replace=False)
        values = df.loc[indices_to_shuffle, col].values
        np.random.shuffle(values)
        df.loc[indices_to_shuffle, col] = values
    return df

def generate_variants(df_ref, levels=[0, 10, 20, 50]):
    datasets = {}
    
    for lvl in levels:
        # 1. OUT DIRECT : Population identique, mélange par colonne
        datasets[f'out_direct_{lvl}'] = apply_shuffle(df_ref, lvl)
        
        # 2. OUT SAMPLE : Tirage avec remise, puis mélange
        # Tirage avec remise de taille N
        df_sample = df_ref.sample(n=len(df_ref), replace=True).reset_index(drop=True)
        datasets[f'out_sample_{lvl}'] = apply_shuffle(df_sample, lvl)
        
    return datasets

variants = generate_variants(df_reference)

## 4. Définition des Indicateurs (Critères G29)

Nous implémentons deux indicateurs principaux[cite: 30, 32]:

1.  **Taux d'Individualisation (Réidentification)** : Capacité à isoler un enregistrement unique dans la base publiée et à le lier correctement à la base de connaissances.
    * *Méthode* : On cherche les lignes uniques sur les quasi-identifiants dans la base publique. Si cette combinaison est unique, on regarde si l'`id_sejour` correspond à celui de la base de référence.
2.  **Taux d'Inférence (Précision)** : Capacité à déduire une valeur sensible (ici `diabete` ou `liste_diag`) à partir des quasi-identifiants.
    * *Méthode* : Si je réidentifie une personne (correctement ou non), quelle est la probabilité que la valeur sensible associée soit la vraie ?

In [ ]:
def calculate_risk(df_connaissance, df_publie, quasi_identifiers, sensitive_col='diabete'):
    """
    Calcule les métriques de risque.
    df_connaissance : Ce que l'attaquant sait (connaissances_externes.txt)
    df_publie : Le fichier attaqué (out_direct ou out_sample)
    qi : Liste des variables connues (scénario)
    """
    # On joint sur les quasi-identifiants
    # Note : Dans out_sample, un individu peut apparaître 0, 1 ou N fois.
    
    # 1. Isoler les combinaisons uniques dans le jeu publié (Individualisation potentielle)
    # L'attaquant cible les lignes qui semblent uniques
    counts = df_publie.groupby(quasi_identifiers).size()
    unique_combinations = counts[counts == 1].index
    
    if len(unique_combinations) == 0:
        return 0.0, 0.0
    
    # Filtrer le dataset publié pour ne garder que les uniques
    # Astuce pandas pour filtrer sur multi-index
    df_pub_unique = df_publie.set_index(quasi_identifiers)
    df_pub_unique = df_pub_unique[df_pub_unique.index.isin(unique_combinations)].reset_index()
    
    # Tentative de lien avec la base de connaissance (Attaque)
    # On regarde si pour ces combinaisons, on retrouve le bon ID
    merged = pd.merge(
        df_pub_unique, 
        df_connaissance, 
        on=quasi_identifiers, 
        how='inner', 
        suffixes=('_pub', '_know')
    )
    
    if len(merged) == 0:
        return 0.0, 0.0
        
    # --- Indicateur 1 : Individualisation Correcte ---
    # Succès si l'ID caché est le même (preuve que c'est le bon séjour)
    # Note : Le sujet dit "l'identifiant... peut servir pour tester si une réidentification a réussi" [cite: 28]
    reid_success = merged[merged['id_sejour_pub'] == merged['id_sejour_know']]
    risk_reid = len(reid_success) / len(df_publie) * 100  # % de la base totale réidentifiée
    
    # --- Indicateur 2 : Inférence Exacte ---
    # Même si l'ID est faux (collision), est-ce que la valeur sensible est la même ?
    # (L'attaquant se fiche de l'ID, il veut savoir si la personne a le diabète)
    inference_success = merged[merged[f'{sensitive_col}_pub'] == merged[f'{sensitive_col}_know']]
    risk_inference = len(inference_success) / len(merged) * 100 # Précision de l'attaque
    
    return risk_reid, risk_inference

## 5. Exécution des Scénarios de "Connaissance Croissante"

Le sujet demande de tester différents scénarios. Nous définissons 3 niveaux :
1.  **Faible** : Âge flou (10 ans), Sexe, Année.
2.  **Moyen** : Âge (5 ans), Sexe, Date (Mois), Spécialité.
3.  **Fort** : Âge exact, Sexe, Date exacte (Jour), Spécialité, Mode entrée.

In [ ]:
scenarios = {
    '1_Faible': ['age10', 'sexe', 'entree_date_y'],
    '2_Moyen': ['age5', 'sexe', 'entree_date_ym', 'specialite'],
    '3_Fort': ['age', 'sexe', 'entree_date_ymd', 'specialite', 'entree_mode']
}

results = []

for name, df_var in variants.items():
    file_type = 'Direct' if 'direct' in name else 'Sample'
    shuffle_level = int(name.split('_')[-1])
    
    for scen_name, cols in scenarios.items():
        r_reid, r_inf = calculate_risk(df_reference, df_var, cols)
        
        results.append({
            'Type': file_type,
            'Mélange (%)': shuffle_level,
            'Scénario': scen_name,
            'Nb_Variables': len(cols),
            'Risque_Reid (%)': r_reid,
            'Risque_Inference (%)': r_inf
        })

df_res = pd.DataFrame(results)
display(df_res.head())

## 6. Présentation des Résultats et Discussion

Nous visualisons l'évolution du risque en fonction du niveau de mélange et du scénario, comme demandé[cite: 33].

In [ ]:
def plot_results(metric, title):
    plt.figure(figsize=(12, 6))
    # Séparation Direct vs Sample via style de ligne ou facet
    sns.lineplot(
        data=df_res, 
        x='Mélange (%)', 
        y=metric, 
        hue='Scénario', 
        style='Type', 
        markers=True, 
        dashes=True
    )
    plt.title(title)
    plt.ylabel(metric)
    plt.ylim(-5, 105)
    plt.show()

# Graphique 1 : Risque de Réidentification (Individualisation)
plot_results('Risque_Reid (%)', 'Risque de Réidentification vs Mélange')

# Graphique 2 : Risque d'Inférence (Précision de l'attribut sensible)
plot_results('Risque_Inference (%)', "Précision de l'Inférence (Diabète) vs Mélange")

### Discussion des résultats [cite: 37]

1.  **Impact du Scénario (Connaissance croissante)** :
    * Plus le nombre et la précision des variables connues augmentent (Scénario Fort), plus le risque de réidentification est élevé (proche de 100% sans mélange).
    * Les variables précises (Date jour vs Année) augmentent drastiquement l'unicité des individus.

2.  **Impact du Mélange (Shuffle)** :
    * Le mélange des colonnes casse les corrélations entre les quasi-identifiants. Le risque de réidentification chute linéairement avec le taux de mélange.
    * Cependant, le risque d'inférence (deviner le diabète) peut rester élevé même si la réidentification exacte chute, car la distribution globale de la variable sensible reste inchangée dans la colonne.

3.  **Direct vs Sample** :
    * Les fichiers `out_sample` (tirage avec remise) présentent généralement un risque plus faible ou plus incertain que `out_direct`.
    * **Raison** : L'échantillonnage introduit une incertitude fondamentale. Un individu de la base de connaissance peut ne pas être dans la base échantillonnée (risque nul), ou y être plusieurs fois (ambiguïté), rendant l'attaque plus difficile que sur la population complète.